In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import APSIMGraphHelpers as AGH
import GraphHelpers as GH
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.dates as mdates
import MathsUtilities as MUte
import shlex # package to construct the git command to subprocess format
import subprocess 
import sqlite3
import scipy.optimize 
from skopt import gp_minimize
from skopt.callbacks import CheckpointSaver
from skopt import load
from skopt.plots import plot_convergence
import matplotlib.gridspec as gridspec
import xmltodict, json

import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
%matplotlib inline

In [2]:
DBFilePath = r"C:/GitHubRepos/PerennialCrops/OrchardOpt.db"
TreeFilePath = r"C:/GitHubRepos/PerennialCrops/OrchardOpt.apsimx"

In [3]:
GrapeSimulationNames = ['Grape_Squire_1Irri60',
 'Grape_Squire_1Irri100',
 'Grape_Squire_2Irri20',
 'Grape_Squire_2Irri100',
 'Grape_Squire_2Irri60',
 'Grape_SeaviewTSysGrape_4Cane',
 'Grape_SeaviewTSysGrape_2Cane',
 'Grape_Seaview_08']
AppleSimulationNames = ['Apple_Tall', 
                        'Apple_Dwarf']
KiwiFruitSimulationNames = ['Kiwifruit']
PeachSimulationNames = ['Peach']

In [4]:
Crops = ['Grape','Apple','KiwiFruit','Peach']

In [5]:
paramNames = ["SurfaceKL",
        "KLReductionFactor",
        "KLReductionDepth",
        "gsmax",
        "r50"]

In [6]:
fittingVars = ['transpiration','psw',
               'Soil.OutputLayers.SW(1)',
               'Soil.OutputLayers.SW(2)',
              'Soil.OutputLayers.SW(3)',
              'Soil.OutputLayers.SW(4)',
              'Soil.OutputLayers.SW(5)',
              'Soil.OutputLayers.SW(6)']

In [7]:
def CalcScaledValue(Value,RMax,RMin):
    return (Value - RMin)/(RMax-RMin)

TreeParamSet = {
          "$type": "Models.Manager, Models",
          "Code": "",
          "Parameters": [],
          "Name": "TreeParamSet",
          "IncludeInDocumentation": False,
          "Enabled": True,
          "ReadOnly": False}

def AppendModeltoModelofTypeAndDeleteOldIfPresent(Parent,TypeToAppendTo,ModelToAppend):
    try:
        for child in Parent['Children']:
            if child['$type'] == TypeToAppendTo:
                pos = 0
                for g in child['Children']:
                    if g['Name'] == ModelToAppend['Name']:
                        del child['Children'][pos]
                        #print('Model ' + ModelToAppend['Name'] + ' found and deleted')
                    pos+=1
                child['Children'].append(ModelToAppend)
                return True
            else:
                Parent = AppendModeltoModelofTypeAndDeleteOldIfPresent(child,TypeToAppendTo,ModelToAppend)
        return Parent
    except:
        return Parent
    
def AppendModeltoModelofType(Parent,TypeToAppendTo,NameToAppendTo,ModelToAppend):
    try:
        for child in Parent['Children']:
            if (child['$type'] == TypeToAppendTo) and (child['Name'] == NameToAppendTo):
                child['Children'].append(ModelToAppend)
                return True
            else:
                Parent = AppendModeltoModelofType(child,TypeToAppendTo,NameToAppendTo,ModelToAppend)
        return Parent
    except:
        return Parent
    
def findNextChild(Parent,ChildName):
    if len(Parent['Children']) >0:
        for child in range(len(Parent['Children'])):
            if Parent['Children'][child]['Name'] == ChildName:
                return Parent['Children'][child]
    else:
        return Parent[ChildName]

def findModel(Parent,PathElements):
    for pe in PathElements:
        Parent = findNextChild(Parent,pe)
    return Parent    

def StopReporting(WheatApsimx,modelPath):
    PathElements = modelPath.split('.')
    report = findModel(WheatApsimx,PathElements)
    report["EventNames"] = []

def removeModel(Parent,modelPath):
    PathElements = modelPath.split('.')
    Parent = findModel(Parent,PathElements[:-1])
    pos = 0
    found = False
    for c in Parent['Children']:
        if c['Name'] == PathElements[-1]:
            del Parent['Children'][pos]
            found = True
            break
        pos += 1
    if found == False:
        print('Failed to find ' + PathElements[-1] + ' to delete')

def ApplyParamReplacementSet(paramValues,paramNames,filePath):
    with open(filePath,'r') as ApsimxJSON:
        Apsimx = json.load(ApsimxJSON)
        ApsimxJSON.close()
    ## Remove old prameterSet manager in replacements
    removeModel(Apsimx,'Replacements.TreeParamSet')

    ## Add crop coefficient overwrite into replacements
    codeString = "using Models.Core;\r\nusing System;\r\nnamespace Models\r\n{\r\n\t[Serializable]\r\n [System.Xml.Serialization.XmlInclude(typeof(Model))]\r\n   public class Script : Model\r\n    {\r\n "
    for p in range(len(paramValues)):
        codeString +=  "     public double "
        codeString += paramNames[p]
        codeString += ' {get; set;} = '
        codeString += str(paramValues[p])
        codeString += '; \r\n'
        
    codeString += '\r\n}\r\n}'

    TreeParamSet["Code"] = codeString

    AppendModeltoModelofType(Apsimx,"Models.Core.Folder, Models","Replacements",TreeParamSet)

    with open(filePath,'w') as ApsimxJSON:
        json.dump(Apsimx,ApsimxJSON,indent=2)
        
def makeLongString(SimulationSet):
    longString =  '/SimulationNameRegexPattern:"'
    longString =  longString + '(' + SimulationSet[0]  + ')|' # Add first on on twice as apsim doesn't run the first in the list
    for sim in SimulationSet[:]:
        longString = longString + '(' + sim + ')|'
    longString = longString + '(' + SimulationSet[-1] + ')|' ## Add Last on on twice as apsim doesnt run the last in the list
    longString = longString + '(' + SimulationSet[-1] + ')"'
    return longString

In [8]:
def Preparefile():
    !del C:\GitHubRepos\PerennialCrops\OrchardOpt.db
    
def runModelItter(paramNames,paramValues,SimulationSet,FittingVariables,TreeFilePath):
    ApplyParamReplacementSet(paramValues,paramNames,TreeFilePath)
    SimSet = makeLongString(SimulationSet)
    #print(SimSet)
    start = dt.datetime.now()
    subprocess.run(['C:/GitHubRepos/ApsimX/bin/Debug/netcoreapp3.1/Models.exe',
                    TreeFilePath,
                    SimSet], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    endrun = dt.datetime.now()
    runtime = (endrun-start).seconds
    con = sqlite3.connect(DBFilePath)
    ObsPred = pd.read_sql("Select * from PredictedObserved",con)
    con.close()
    n = len(ObsPred.loc[:,'SimulationID'].drop_duplicates())        
    
    ScObsPre = pd.DataFrame(columns = ['ScObs','ScPred','Var'])
    indloc = 0
    for var in FittingVariables:
        DataPairs = ObsPred.reindex(['Observed.'+var,'Predicted.'+var],axis=1).dropna()
        for v in ['Observed.'+var,'Predicted.'+var]:
            DataPairs.loc[:,v] = pd.to_numeric(DataPairs.loc[:,v])
        VarMax = max(DataPairs.loc[:,'Observed.'+var].max(),DataPairs.loc[:,'Predicted.'+var].max())
        VarMin = min(DataPairs.loc[:,'Observed.'+var].min(),DataPairs.loc[:,'Predicted.'+var].min())
        for x in DataPairs.index:
            ScObsPre.loc[indloc,'ScObs'] = CalcScaledValue(DataPairs.loc[x,'Observed.'+var],VarMax,VarMin)
            ScObsPre.loc[indloc,'ScPred'] = CalcScaledValue(DataPairs.loc[x,'Predicted.'+var],VarMax,VarMin)
            ScObsPre.loc[indloc,'Var'] = var
            indloc+=1
    RegStats = MUte.MathUtilities.CalcRegressionStats('LN',ScObsPre.loc[:,'ScPred'].values,ScObsPre.loc[:,'ScObs'].values)
    try:
        retVal = max(RegStats.NSE,-2) *-1
        print(str(globals()['itter']) +"  "+str(paramValues) + " run completed " +str(n) + ' sims in '+ str(runtime) + ' seconds.  NSE = '+str(RegStats.NSE))
    except:
        retVal = 2
        print(str(globals()['itter']) +"  "+ str(paramValues) + " run completed " +str(n) + ' sims in '+ str(runtime) + ' seconds.  NSE = insufficient data')
    globals()['itter'] +=1
    return retVal

def runDevModelItter(paramValues):
    retVal =runModelItter(paramNames,paramValues,SimulationNames,fittingVars,TreeFilePath)
    return retVal

In [12]:
# SimulationSet = PeachSimulationNames
# SimSet = makeLongString(SimulationSet)
# subprocess.run(['C:/GitHubRepos/ApsimX/bin/Debug/netcoreapp3.1/Models.exe',
#                 TreeFilePath,
#                 SimSet], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

CompletedProcess(args=['C:/GitHubRepos/ApsimX/bin/Debug/netcoreapp3.1/Models.exe', 'C:/GitHubRepos/PerennialCrops/OrchardOpt.apsimx', '/SimulationNameRegexPattern:"(Peach)|(Peach)|(Peach)|(Peach)"'], returncode=0, stdout=b'')

In [9]:
["SurfaceKL",
"KLReductionFactor",
"KLReductionDepth",
"gsmax",
"r50"]

['SurfaceKL', 'KLReductionFactor', 'KLReductionDepth', 'gsmax', 'r50']

In [11]:
for crop in Crops:
    globals()['SimulationNames'] = globals()[crop+'SimulationNames']
    Preparefile()
    checkpoint_saver = CheckpointSaver(crop+" FitsCheckpoint.pkl", compress=9)
    RandomCalls = 100
    OptimizerCalls =30
    TotalCalls = RandomCalls + OptimizerCalls
    x0 = [0.1,0.002,250,0.009,150]
    bounds = [(0.05,0.3),
              (0.002,0.007),
              (230,270),
              (0.001,0.015),
              (100,200)]
    globals()['itter'] = 0
    ret = gp_minimize(runDevModelItter, bounds, n_calls=TotalCalls,n_initial_points=RandomCalls,
                 initial_point_generator='sobol',callback=[checkpoint_saver],x0=x0)

C:\Users\Cflhxb\AppData\Local\anaconda3\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn("The balance properties of Sobol' points require "


0  [0.1, 0.002, 250, 0.009, 150] run completed 8 sims in 13 seconds.  NSE = 0.6117688223167475
1  [0.25459267641532307, 0.003869476384166938, 256, 0.005230017300821034, 196] run completed 8 sims in 23 seconds.  NSE = 0.6013365933345609
2  [0.06709267641532309, 0.002619476384166938, 266, 0.0017300173008210338, 121] run completed 8 sims in 25 seconds.  NSE = 0.14322216079911254
3  [0.19209267641532307, 0.005119476384166938, 246, 0.008730017300821034, 171] run completed 8 sims in 20 seconds.  NSE = 0.5924885825473565
4  [0.22334267641532307, 0.0032444763841669383, 261, 0.013980017300821032, 133] run completed 8 sims in 21 seconds.  NSE = 0.4627351726527934
5  [0.09834267641532309, 0.005744476384166938, 241, 0.006980017300821034, 183] run completed 8 sims in 22 seconds.  NSE = 0.5856695388085967
6  [0.28584267641532307, 0.006994476384166938, 251, 0.0034800173008210336, 158] run completed 8 sims in 24 seconds.  NSE = 0.5114256398588296
7  [0.16084267641532307, 0.0044944763841669386, 231, 0.

C:\Users\Cflhxb\AppData\Local\anaconda3\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn("The balance properties of Sobol' points require "


0  [0.1, 0.002, 250, 0.009, 150] run completed 2 sims in 12 seconds.  NSE = 0.7020685507898927
1  [0.1930327023239008, 0.0023572158216245438, 232, 0.014407437005692119, 106] run completed 2 sims in 11 seconds.  NSE = 0.1959551313578355
2  [0.2555327023239008, 0.006107215821624543, 242, 0.01090743700569212, 131] run completed 2 sims in 12 seconds.  NSE = 0.5965508968507702
3  [0.1305327023239008, 0.0036072158216245436, 262, 0.003907437005692121, 181] run completed 2 sims in 13 seconds.  NSE = 0.06480537967402733
4  [0.1617827023239008, 0.006732215821624543, 237, 0.009157437005692118, 144] run completed 2 sims in 12 seconds.  NSE = 0.7115628012182622
5  [0.2867827023239008, 0.004232215821624544, 257, 0.002157437005692121, 194] run completed 2 sims in 12 seconds.  NSE = -0.5396485358972833
6  [0.2242827023239008, 0.005482215821624543, 267, 0.012657437005692117, 169] run completed 2 sims in 11 seconds.  NSE = 0.5706336988053029
7  [0.09928270232390081, 0.0029822158216245434, 247, 0.0056574

C:\Users\Cflhxb\AppData\Local\anaconda3\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn("The balance properties of Sobol' points require "


0  [0.1, 0.002, 250, 0.009, 150] run completed 1 sims in 13 seconds.  NSE = 0.6543512727499798
1  [0.1937777544927013, 0.003982297286331627, 268, 0.004676922937395746, 185] run completed 1 sims in 18 seconds.  NSE = 0.49633714332493395
2  [0.2562777544927013, 0.0027322972863316267, 238, 0.0011769229373957467, 110] run completed 1 sims in 21 seconds.  NSE = -0.060233391125843916
3  [0.1312777544927013, 0.005232297286331627, 258, 0.008176922937395746, 160] run completed 1 sims in 18 seconds.  NSE = 0.6699517089676162
4  [0.1625277544927013, 0.0033572972863316272, 233, 0.013426922937395745, 122] run completed 1 sims in 19 seconds.  NSE = -0.08915717222217934
5  [0.2875277544927013, 0.005857297286331627, 253, 0.006426922937395746, 172] run completed 1 sims in 14 seconds.  NSE = 0.6523380494769943
6  [0.2250277544927013, 0.002107297286331627, 263, 0.0029269229373957463, 147] run completed 1 sims in 12 seconds.  NSE = 0.2265767576252109
7  [0.10002775449270128, 0.004607297286331627, 243, 0.0

C:\Users\Cflhxb\AppData\Local\anaconda3\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn("The balance properties of Sobol' points require "


0  [0.1, 0.002, 250, 0.009, 150] run completed 1 sims in 8 seconds.  NSE = 0.4247461980171735
1  [0.06813173932921608, 0.005536180443236277, 252, 0.003941189442026141, 193] run completed 1 sims in 6 seconds.  NSE = 0.488724491982647
2  [0.13063173932921607, 0.004286180443236276, 262, 0.014441189442026141, 118] run completed 1 sims in 8 seconds.  NSE = 0.5217832563991998
3  [0.2556317393292161, 0.006786180443236277, 242, 0.00744118944202614, 168] run completed 1 sims in 11 seconds.  NSE = 0.5710569245741712
4  [0.2868817393292161, 0.004911180443236277, 257, 0.01269118944202614, 130] run completed 1 sims in 7 seconds.  NSE = 0.4156614148849387
5  [0.16188173932921607, 0.0024111804432362764, 237, 0.00569118944202614, 180] run completed 1 sims in 7 seconds.  NSE = 0.5186350102118371
6  [0.09938173932921608, 0.0036611804432362767, 247, 0.0021911894420261406, 155] run completed 1 sims in 7 seconds.  NSE = 0.36375601702348714
7  [0.22438173932921612, 0.006161180443236276, 267, 0.0091911894420

In [18]:
Apple = load("Apple FitsCheckpoint.pkl")

In [19]:
Apple

          fun: -0.6428777504782437
            x: [0.25673473884283743, 0.01, 100, 0.01449734785009412, 287]
    func_vals: [-3.251e-01  2.377e-01 ... -5.840e-01 -5.846e-01]
      x_iters: [[0.22, 0.001, 150, 0.009, 200], [0.38757945484501416, 0.004045111160238179, 265, 0.00176221137443572, 82], [0.05007945484501417, 0.0015576111602381797, 365, 0.012262211374435717, 145], [0.27507945484501417, 0.006532611160238179, 165, 0.0052622113744357196, 270], [0.3313294548450142, 0.00280136116023818, 315, 0.010512211374435719, 176], [0.10632945484501417, 0.007776361160238179, 115, 0.0035122113744357197, 51], [0.4438294548450142, 0.0003138611602381797, 215, 0.014012211374435719, 238], [0.2188294548450142, 0.005288861160238179, 415, 0.007012211374435719, 113], [0.24695445484501416, 0.00217948616023818, 190, 0.00438721137443572, 98], [0.4719544548450142, 0.007154486160238179, 390, 0.01138721137443572, 223], [0.13445445484501417, 0.009641986160238179, 290, 0.00788721137443572, 285], [0.35945445484501

In [ ]:
peach fun: -0.5399897583440487          x: [0.31605349182051806, 0.009985811208077913, 109, 0.013001374918556696, 81]

In [11]:
Peach = ret

In [10]:
ret

          fun: -0.5399897583440487
            x: [0.31605349182051806, 0.009985811208077913, 109, 0.013001374918556696, 81]
    func_vals: [ 3.081e-02 -4.029e-01 ... -5.017e-01 -5.098e-01]
      x_iters: [[0.22, 0.001, 150, 0.009, 200], [0.17894411682051808, 0.008197920583077913, 369, 0.010048249918556695, 67], [0.2914441168205181, 0.005710420583077912, 469, 0.006548249918556694, 130], [0.06644411682051808, 0.0007354205830779137, 269, 0.013548249918556694, 255], [0.12269411682051808, 0.006954170583077912, 419, 0.004798249918556695, 161], [0.3476941168205181, 0.001979170583077914, 219, 0.011798249918556693, 286], [0.2351941168205181, 0.004466670583077912, 319, 0.008298249918556693, 224], [0.4601941168205181, 0.009441670583077913, 119, 0.0012982499185566967, 99], [0.4883191168205181, 0.006332295583077912, 294, 0.012673249918556694, 83], [0.26331911682051806, 0.0013572955830779135, 494, 0.005673249918556694, 208], [0.37581911682051805, 0.003844795583077913, 394, 0.0021732499185566963, 27

In [ ]:
Cultivars = ['Whistler']

In [ ]:
Version = '5'
#Kappas=pd.Series(index=[0,1,2],data=[100,10,1])
FailedCultivars = []
rounds = 0
while (rounds<3):
    for c in Cultivars:
        print ('Fitting '+c)
        try:
            checkpoint_saver = CheckpointSaver("./OptFiles/"+c+" FitsCheckpoint"+Version+".pkl", compress=9)
            Preparefile(WheatFilePath,'Harvest')
            RandomCalls = 50
            OptimizerCalls =15
            TotalCalls = RandomCalls + OptimizerCalls
            try:
                x0 = list(CampInputs.loc[c,paramNames].values)
            except:
                x0 = [8.5, 10.2, 7.0, 4.0, 120, 4.5]
            bounds = [(5.0,15.0),
                      (0.0,12.0),
                      (0.0,12.0),
                      (-10.0,10.0),
                      (50,500),
                      (0,8.0)]

            if rounds == 0:
                print ('Creating empty IttersObsPred dataframe')
                globals()['IttersObsPred'] = pd.DataFrame(columns = ['ScObs','ScPred','Var','Predicted.Simulation.Name','Predicted.Experiment','Predicted.Wheat.SowingDate',
                                                       'NSE','nSims']+paramNames,
                                                        index=pd.MultiIndex.from_arrays([[],[],[]],names=['Cultivar','itter','indloc']))
                globals()['itter'] = 0

                ret = gp_minimize(runDevModelItter, bounds, n_calls=TotalCalls,n_initial_points=RandomCalls,
                         initial_point_generator='sobol',callback=[checkpoint_saver],x0=x0)
                currentBest = ret.fun
            else:
                CheckPoint = load("./OptFiles/"+c+" FitsCheckpoint"+Version+".pkl")
                globals()['IttersObsPred'] = pd.read_pickle("./OptFiles/"+c+"_IttersObsPred"+Version+".pkl")
                globals()['itter'] = len(CheckPoint.func_vals)
                x0 = CheckPoint.x_iters
                y0 = CheckPoint.func_vals
                ret = gp_minimize(runDevModelItter, bounds, n_calls=TotalCalls,n_initial_points=RandomCalls,
                         initial_point_generator='sobol',callback=[checkpoint_saver],x0=x0,y0=y0)
                currentBest = ret.fun
            print(c)
            print(str(rounds))
            print(str(currentBest))
        except:
            FailedCultivars.append(c)
    rounds += 1

In [ ]:
def PlotObsPre(obsPreSet,c,ret):
    graph = plt.figure(figsize=(10,20))
    gs = gridspec.GridSpec(6, 6)
    ax = graph.add_subplot(gs[0, 0:2])
    colors = pd.Series(index = ['Wheat.Phenology.FinalLeafNumber','Wheat.Phenology.FlagLeafDAS',
                        'Wheat.Phenology.HeadingDAS','Wheat.Phenology.FloweringDAS'],
                       data = ['r','b','g','k'])
    markers = ['o','s','^','v','1','2','3','4','+','x','d']

    for var in obsPreSet.loc[:,'Var'].drop_duplicates():
        epos = 0
        lablab = var.replace('Wheat.Phenology.','')
        for expt in obsPreSet.loc[:,'Predicted.Experiment'].drop_duplicates():
            filt = ((obsPreSet.loc[:,'Var'] == var) & (obsPreSet.loc[:,'Predicted.Experiment'] == expt))
            plt.plot(obsPreSet.loc[filt,'ScObs'],obsPreSet.loc[filt,'ScPred'],
                     markers[epos],color=colors[var],label=lablab)
            
            epos+=1
            if epos == 11:
                epos= 0
            lablab = None
    plt.plot([0,1],[0,1],'-',color='k')
    plt.ylabel('Scalled Predictions')
    plt.xlabel('Scalled Observations')
    plt.legend(bbox_to_anchor=(1.1, 1.05))
    RegStats = MUte.MathUtilities.CalcRegressionStats('LN',obsPreSet.loc[:,'ScPred'].values,obsPreSet.loc[:,'ScObs'].values)
    plt.text(0.97,0.03,'NSE = '+str(RegStats.NSE)[:4],transform=ax.transAxes,horizontalalignment='right')
    plt.text(0.03,0.97,c,horizontalalignment='left')
    
    ax = graph.add_subplot(gs[0, 3:6])
    plot_convergence(ret);
    plt.ylim(-1,0)
    #plt.plot([20,20,35,35,55,55,70,70,90,90,105,105,125,125,140,140,155,155,175,175,190,190,210,210,225,225]
    
    ShortParams = pd.Series(index=paramNames,data=['MinLN','PpLN','VrnLN','VxPLN','LDB','HPPS'])
    pos = 0
    for p in paramNames:
        ax = graph.add_subplot(gs[1,pos])
        plt.plot(ParamCombs.loc[:,p],-ParamCombs.loc[:,'NSE'],'o',color='k')
        bestFit = ParamCombs.loc[:,'NSE'].idxmin()
        plt.plot(ParamCombs.loc[bestFit,p],-ParamCombs.loc[bestFit,'NSE'],'o',color='cyan',ms=8,mec='k',mew=2)
        plt.plot(ret.x_iters[0][pos],-ret.func_vals[0],'o',color='orange',ms=8,mec='k',mew=2)
        if pos == 0:
            plt.ylabel('NSE')
        else:
            ax.axes.yaxis.set_visible(False)
        pos+=1
        plt.xlabel(ShortParams[p])
        plt.ylim(0,1)
    plt.tight_layout()

In [ ]:
c = 'Bennett'
IttersObsPred = pd.read_pickle("./OptFiles/"+c+"_IttersObsPred"+Version+".pkl")
ret = load("./OptFiles/"+c+" FitsCheckpoint"+Version+".pkl")
ParamCombs = pd.DataFrame(ret.x_iters,columns = paramNames)
ParamCombs.loc[:,'NSE'] = ret.func_vals
bestFitItter = ParamCombs.loc[:,'NSE'].idxmin()
bestFitObsPred = IttersObsPred.loc[(c,bestFitItter),:]
PlotObsPre(bestFitObsPred,c,ret)

In [ ]:
from skopt.plots import plot_objective
plot_objective(ret)#,minimum='expected_minimum')